## Part 1

    1. Read the case.csv file from the 311 call data into a Spark DataFrame.
    2. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
    3. How many Stray Animal cases are there?
    4. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?
    5. Create a new DataFrame without any information related to dates or location.
    6. Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".

In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr

In [2]:
spark = SparkSession.builder.master("local").appName("read").\
    enableHiveSupport().\
    getOrCreate()

In [3]:
df = spark.read.csv('./case.csv', header=True)

In [4]:
type(df)

pyspark.sql.dataframe.DataFrame

In [5]:
df.head(5)

[Row(case_id='1014127332', case_opened_date='1/1/18 0:42', case_closed_date='1/1/18 12:29', SLA_due_date='9/26/20 0:42', case_late='NO', num_days_late='-998.5087616000001', case_closed='YES', dept_division='Field Operations', service_request_type='Stray Animal', SLA_days='999.0', case_status='Closed', source_id='svcCRMLS', request_address='2315  EL PASO ST, San Antonio, 78207', council_district='5'),
 Row(case_id='1014127333', case_opened_date='1/1/18 0:46', case_closed_date='1/3/18 8:11', SLA_due_date='1/5/18 8:30', case_late='NO', num_days_late='-2.0126041669999997', case_closed='YES', dept_division='Storm Water', service_request_type='Removal Of Obstruction', SLA_days='4.322222222', case_status='Closed', source_id='svcCRMSS', request_address='2215  GOLIAD RD, San Antonio, 78223', council_district='3'),
 Row(case_id='1014127334', case_opened_date='1/1/18 0:48', case_closed_date='1/2/18 7:57', SLA_due_date='1/5/18 8:30', case_late='NO', num_days_late='-3.022337963', case_closed='YES',

In [6]:
df.orderBy(df.case_opened_date.desc()).where(df.case_closed == 'NO').show()

+----------+----------------+----------------+--------------+---------+-------------------+-----------+--------------------+--------------------+------------------+-----------+------------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|  SLA_due_date|case_late|      num_days_late|case_closed|       dept_division|service_request_type|          SLA_days|case_status|   source_id|     request_address|council_district|
+----------+----------------+----------------+--------------+---------+-------------------+-----------+--------------------+--------------------+------------------+-----------+------------+--------------------+----------------+
|1013863552|    9/9/17 11:43|            null| 11/14/17 8:30|      YES|        47.64583333|         NO|    Code Enforcement|    Permits Building|       65.86542824|       Open|CRM_Listener|905  PRADO ST, Sa...|               5|
|1013860368|     9/8/17 9:47|            null|  1/26/18 9:47|       NO|       -25.408229

How many Stray Animal cases are there?

In [7]:
df.select('service_request_type').where(df.service_request_type == 'Stray Animal').count()

26760

How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [8]:
(df
 .select('dept_division')
 .where(df.dept_division == 'Field Operations')
 .where(df.service_request_type != 'Officer Standby')
 .count())

113902

Create a new DataFrame without any information related to dates or location.

In [9]:
df_no_dates = df.drop('case_opened_date', 'case_closed_date', 'SLA_due_date', 'request_address', 'council_district')

In [10]:
df_no_dates.show(5)

+----------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+
|   case_id|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|
+----------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+
|1014127332|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|
|1014127333|       NO|-2.0126041669999997|        YES|     Storm Water|Removal Of Obstru...|4.322222222|     Closed| svcCRMSS|
|1014127334|       NO|       -3.022337963|        YES|     Storm Water|Removal Of Obstru...|4.320729167|     Closed| svcCRMSS|
|1014127335|       NO|       -15.01148148|        YES|Code Enforcement|Front Or Side Yar...|16.29188657|     Closed| svcCRMSS|
|1014127336|      YES|0.37216435200000003|        YES|Field Operations|Animal Cruelty(Cr...|      0.125|     Cl

Read dept.csv into a Spark DataFrame. Inspect the dept_name column. Replace the missing values with "other".

In [11]:
df = spark.read.csv('./dept.csv', header=True)

In [12]:
df.show()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|                null|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

In [13]:
df = df.na.fill('Other', ['dept_name']).show()

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
|    Code Enforcement|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Code Enforcement ...|               Other|  DSD/Code Enforcement|                YES|
|   Dangerous Premise|Code Enforcement ...|  DSD/Code Enforcement|                YES|
|Dangerous Premise...|Code Enforcement ...|

## Part 2

    1. Convert the council_district column to a string column.
    2. Extract the year from the case_closed_date column.
    3. Convert num_days_late from days to hours in new columns num_hours_late.
    4. Convert the case_late column to a boolean column.
    5. Convert the SLA_days columns to a double column.
    6. Pull it all together

In [31]:
df = spark.read.csv('./case.csv', header=True)

Convert the council_district column to a string column.

In [33]:
df.council_district.cast('string')

Column<b'CAST(council_district AS STRING)'>

In [34]:
df.dtypes

[('case_id', 'string'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'string'),
 ('case_closed', 'string'),
 ('dept_division', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'string'),
 ('case_status', 'string'),
 ('source_id', 'string'),
 ('request_address', 'string'),
 ('council_district', 'string')]

In [35]:
df.show()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|
|1014127333|     1/1/18 0:46|     1/3/18 8:11| 1/5/18 8:30|       NO|-2.0126041669999997|        YES|     Storm Water|Remova

In [37]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

Extract the year from the case_closed_date column.

In [65]:
df_time = df.select(to_timestamp(df.case_closed_date, 'M/d/y H:mm').cast('string').alias('time'))

In [67]:
df_time.select(df_time.time, substring(df_time.time, 0,4)).show()

+-------------------+---------------------+
|               time|substring(time, 0, 4)|
+-------------------+---------------------+
|2018-01-01 12:29:00|                 2018|
|2018-01-03 08:11:00|                 2018|
|2018-01-02 07:57:00|                 2018|
|2018-01-02 08:13:00|                 2018|
|2018-01-01 13:29:00|                 2018|
|2018-01-01 14:38:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:32:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|                 2018|
|2018-01-02 15:33:00|           

Convert num_days_late from days to hours in new columns num_hours_late.

In [70]:
df.withColumn('num_hours_late', df.num_days_late*24).show()

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     -23964.2102784|
|1014127333|     1/1/18 0:46|     1/3/18 8:1

Convert the case_late column to a boolean column.

In [79]:
df.select('case_id', 'case_late').withColumn('case_late', df.case_late.cast('boolean')).show()

+----------+---------+
|   case_id|case_late|
+----------+---------+
|1014127332|    false|
|1014127333|    false|
|1014127334|    false|
|1014127335|    false|
|1014127336|     true|
|1014127337|    false|
|1014127338|    false|
|1014127339|    false|
|1014127340|    false|
|1014127341|    false|
|1014127342|    false|
|1014127343|    false|
|1014127344|    false|
|1014127345|    false|
|1014127346|    false|
|1014127347|    false|
|1014127348|    false|
|1014127349|    false|
|1014127350|    false|
|1014127351|    false|
+----------+---------+
only showing top 20 rows



Convert the SLA_days columns to a double column.

In [80]:
df.select('SLA_days').show()

+------------------+
|          SLA_days|
+------------------+
|             999.0|
|       4.322222222|
|       4.320729167|
|       16.29188657|
|             0.125|
|       30.08446759|
|       16.06429398|
|16.063796300000003|
|       16.06333333|
|        16.0628588|
|       16.06237269|
|       16.06104167|
|       16.06059028|
|       16.06011574|
|       16.05953704|
|       16.05907407|
|       16.05864583|
|       16.05819444|
|       16.05775463|
|       16.05733796|
+------------------+
only showing top 20 rows



In [82]:
SLA_double = df.select('SLA_days').withColumn('SLA_days', df.SLA_days.cast('double'))

In [84]:
SLA_double.dtypes

[('SLA_days', 'double')]

Pull it all together

In [85]:
(df
 .withColumn('num_hours_late', df.num_days_late*24)
 .withColumn('case_late', df.case_late.cast('boolean'))
 .withColumn('SLA_days', df.SLA_days.cast('double'))
 .show())

+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+
|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|          SLA_days|case_status|source_id|     request_address|council_district|     num_hours_late|
+----------+----------------+----------------+------------+---------+-------------------+-----------+----------------+--------------------+------------------+-----------+---------+--------------------+----------------+-------------------+
|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|    false| -998.5087616000001|        YES|Field Operations|        Stray Animal|             999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|               5|     -23964.2102784|
|1014127333|     1/1/18 0:46|     1/3/18 8:1

## Part 3